In [7]:
# from google.colab import drive
# drive.mount('/content/drive')
import json
import sys

In [10]:
sys.path.append('/home/taindp/PycharmProjects/thesis/dqn')

In [1]:
%pwd

'/home/taindp/PycharmProjects/thesis/dqn'

In [11]:
# %cd /content/drive/My\ Drive/DQN
# FOLDER_PATH = '/home/taindp/Jupyter/custom_dqn'
# FOLDER_PATH = '/home/taindp/PycharmProjects/thesis/dqn'
FOLDER_PATH = %pwd
# FOLDER_PATH = '/home/waves8/taindp/deep_q_learning'
CONSTANTS_FILE_PATH = f'{FOLDER_PATH}/constants.json'
# SAVE_WEIGHT_PATH =f'weight/goal400_10.h5'
%cd $FOLDER_PATH
!ls

/home/taindp/PycharmProjects/thesis/dqn
check.ipynb		   LICENSE		train.py
constants.json		   log3.txt		userold.py
create_goal.ipynb	   pickle_converter.py	user.py
db_query.py		   __pycache__		user_simulator.py
dialogue_config.py	   state_tracker.py	utils.py
dqn_agent.py		   test.py		weight
error_model_controller.py  train_dqn.ipynb


In [12]:
%load_ext autoreload
%autoreload 2
from user_simulator import UserSimulator
from error_model_controller import ErrorModelController
from dqn_agent import DQNAgent
# from dqn_agent import get_action
from state_tracker import StateTracker
import pickle, argparse, json, math
from utils import remove_empty_slots
from user import User
import time
import json
from tqdm import tqdm
from db_query import DBQuery
from datetime import date
today = str(date.today().strftime("%d/%m/%Y")).replace(r'/',r'_')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'dqn'

In [4]:
def run_round(state, warmup=False):
    # 1) Agent takes action given state tracker's representation of dialogue (state)
    agent_action_index, agent_action = dqn_agent.get_action(state, use_rule=warmup)
#     print('agent_action_index',agent_action_index)
#     print('agent_action',agent_action)
    # 2) Update state tracker with the agent's action
    state_tracker.update_state_agent(agent_action)
#     print('current round',agent_action['round'])
    # 3) User takes action given agent action
    user_action, reward, done, success = user.step(agent_action)
#     print('user_action',user_action)
#     print('reward',reward)
#     print('done',done)
#     print('success',success)
    if not done:
        # 4) Infuse error into semantic frame level of user action
        emc.infuse_error(user_action)
#         print('user_action emc',user_action)
    # 5) Update state tracker with user action
    state_tracker.update_state_user(user_action)
#     print('update_state_user',user_action)
    # 6) Get next state and add experience
    next_state = state_tracker.get_state(done)
#     print('next_state',next_state)
    dqn_agent.add_experience(state, agent_action_index, reward, next_state, done)
    return next_state, reward, done, success


def warmup_run():
    """
    Runs the warmup stage of training which is used to fill the agents memory.

    The agent uses it's rule-based policy to make actions. The agent's memory is filled as this runs.
    Loop terminates when the size of the memory is equal to WARMUP_MEM or when the memory buffer is full.

    """

    print('Warmup Started...')
    total_step = 0
    with tqdm(total=1) as pbar:
        while total_step != WARMUP_MEM and not dqn_agent.is_memory_full():
            # Reset episode
            episode_reset()
            done = False
            # Get initial state from state tracker
            state = state_tracker.get_state()
            # print('State warmup', state)
            # print('Type state',type(state))
            while not done:
                next_state, _, done, _ = run_round(state, warmup=True)
#                 next_state, _, done, _ = run_round(state)
                total_step += 1
                state = next_state
            # print(total_step)
            # time.sleep(1)
            pbar.update(1)
    print('...Warmup Ended')


def train_run():
    """
    Runs the loop that trains the agent.

    Trains the agent on the goal-oriented chatbot task. Training of the agent's neural network occurs every episode that
    TRAIN_FREQ is a multiple of. Terminates when the episode reaches NUM_EP_TRAIN.

    """

    print('Training Started...')
    episode = 0
    period_reward_total = 0
    period_success_total = 0
    success_rate_best = 0.6
    # with tqdm(total=NUM_EP_TRAIN) as pbar1:
    while episode < NUM_EP_TRAIN:
        episode_reset()
        episode += 1
#         print('Current episode:',episode)
        done = False
        state = state_tracker.get_state()
        # print('Current state',state)
        while not done:
            next_state, reward, done, success = run_round(state)
            # print('Next state',next_state)
#             print('Reward', reward)
#             print('Done',done)
            period_reward_total += reward
            state = next_state
#         print("success :{0}".format(success))

        period_success_total += success
        # print('period_success_total',period_success_total)
        # Train
        if episode % TRAIN_FREQ == 0:
            print('Current episode:',episode)
            # Check success rate
            success_rate = period_success_total / TRAIN_FREQ
            avg_reward = period_reward_total / TRAIN_FREQ
            print("episode {0}: success rate: {1}".format(episode, success_rate))

            # Flush
            if success_rate >= success_rate_best and success_rate >= SUCCESS_RATE_THRESHOLD:
                dqn_agent.empty_memory()
            # Update current best success rate
            if success_rate >= success_rate_best:
                print('Episode: {} NEW BEST SUCCESS RATE: {} Avg Reward: {}' .format(episode, success_rate, avg_reward))
                success_rate_best = success_rate
                dqn_agent.save_weights()
            period_success_total = 0
            period_reward_total = 0
            # Copy
            dqn_agent.copy()
            # Train
            dqn_agent.train()
          # pbar1.update(1)
    print('...Training Ended')


def episode_reset():
    """
    Resets the episode/conversation in the warmup and training loops.

    Called in warmup and train to reset the state tracker, user and agent. Also get's the initial user action.

    """

    # First reset the state tracker
    state_tracker.reset()
    # Then pick an init user action
    user_action = user.reset()
    # Infuse with error
    emc.infuse_error(user_action)
    # And update state tracker
    state_tracker.update_state_user(user_action)
    # Finally, reset agent
    dqn_agent.reset()

In [5]:
constants_file = CONSTANTS_FILE_PATH

with open(constants_file) as f:
    constants = json.load(f)

NameError: name 'json' is not defined

In [ ]:
constants

In [ ]:
# Load file path constants
file_path_dict = constants['db_file_paths']
DATABASE_FILE_PATH = file_path_dict['database']
DICT_FILE_PATH = file_path_dict['dict']
USER_GOALS_FILE_PATH = file_path_dict['user_goals']

In [ ]:
run_dict = constants['run']
USE_USERSIM = run_dict['usersim']
WARMUP_MEM = run_dict['warmup_mem']
NUM_EP_TRAIN = run_dict['num_ep_run']
TRAIN_FREQ = run_dict['train_freq']
MAX_ROUND_NUM = run_dict['max_round_num']
SUCCESS_RATE_THRESHOLD = run_dict['success_rate_threshold']

In [ ]:
WARMUP_MEM

In [ ]:
NUM_EP_TRAIN

In [ ]:
constants['agent']['save_weights_file_path'] = '/home/taindp/PycharmProjects/thesis/dqn/weight/' + str(today)+ '_' +str(NUM_EP_TRAIN)+ '_'+str(TRAIN_FREQ)+'_'+str(MAX_ROUND_NUM) + '.h5'

In [ ]:
# constants

In [ ]:
database= json.load(open(DATABASE_FILE_PATH,encoding='utf-8'))

In [ ]:
db_dict = json.load(open(DICT_FILE_PATH,encoding='utf-8'))[0]

In [ ]:
user_goals = json.load(open(USER_GOALS_FILE_PATH,encoding='utf-8'))

In [ ]:
if USE_USERSIM:
    user = UserSimulator(user_goals, constants, database)
else:
    user = User(constants)
emc = ErrorModelController(db_dict, constants)
state_tracker = StateTracker(database, constants)
dqn_agent = DQNAgent(state_tracker.get_state_size(), constants)

In [ ]:
warmup_run()

In [ ]:
train_run()

In [ ]:
# constants['agent']['save_weights_file_path'] = f'weight/goal400_100_10k.h5'
# dqn_agent = DQNAgent(state_tracker.get_state_size(), constants)

In [ ]:
constants

In [ ]:
dqn_agent.save_weights()

In [ ]:
# NUM_EP_TEST = run_dict['num_ep_run']

In [ ]:
NUM_EP_TEST = 500

In [ ]:
def test_run():
    """
    Runs the loop that tests the agent.

    Tests the agent on the goal-oriented chatbot task. Only for evaluating a trained agent. Terminates when the episode
    reaches NUM_EP_TEST.

    """

    print('Testing Started...')
    episode = 0
    while episode < NUM_EP_TEST:
        episode_reset()
        episode += 1
        ep_reward = 0
        done = False
        # Get initial state from state tracker
        state = state_tracker.get_state()
        while not done:
            # Agent takes action given state tracker's representation of dialogue
            agent_action_index, agent_action = dqn_agent.get_action(state)
            # Update state tracker with the agent's action
            state_tracker.update_state_agent(agent_action)
            print("agent: {}".format(str(agent_action)))

            # User takes action given agent action
            user_action, reward, done, success = user.step(agent_action)
            print("user: {}".format(str(user_action)))

            ep_reward += reward
            if not done:
                # Infuse error into semantic frame level of user action
                emc.infuse_error(user_action)
            # Update state tracker with user action
            state_tracker.update_state_user(user_action)
            # Grab "next state" as state
            state = state_tracker.get_state(done)
        print('Episode: {} Success: {} Reward: {}'.format(episode, success, ep_reward))
    print('...Testing Ended')

In [ ]:
test_run()